In [1]:
import functions as fct
import pandas as pd
import dynamics as dyn
import numpy as np
import dask
from cluster import new_launch_cluster
import graphics as gx

# parameter setting
nb_banks=50
path = "./results/parameter_space/test/"
initial_deposits_size = 40

# reg ratios
alpha_init=False  # initial cash (< 1/(1-gamma) - beta)
alpha=0.01
beta_init=0.5 # initial collateral  (< 1/(1-gamma) - alpha)
beta_reg=0.5
beta_star=0.5
gamma=0.03
gamma_init = 3*gamma
collateral_value=1.0

# initialisation of deposits size
initialization_method="pareto"
alpha_pareto=1.3

# shocks on deposits 
shocks_method="non-conservative"
shocks_law="normal-mean-reverting"
shocks_vol=0.07 # 0.05

# speed of learning
learning_speed = 0.5

# min trans size
min_repo_trans_size=1e-8  # 1e-8

# dynamics & ploting
nb_steps=int(10e3)
dump_period=int(5e3)
plot_period=int(5e3)
cp_option=True
heavy_plot=False

# LCR mgt
LCR_mgt_opt=True

# leverage mgt
end_repo_period=False # if int, periodic end repo / if false, leverage mgt
gamma_star = 1.5*gamma
check_leverage_opt = False # to avoid killing the run if one or several banks are below min leverage due to high shocks (there is not possibility of decrease balance sheet size if no interbank borrowings)

# money creation
loan_tenor=nb_steps # if int, money creation / if false. no new loans
loan_period=1
new_loans_vol = 0 # standard deviation around the mean creation of loans
new_loans_mean = 2e-2/250 # daily mean increase in loans expressed as a percentage of the intital loans (the other option would be a relative increase but it is unstable np.exp(np.log(1.01)/250)-1 = 4e-5)
beta_new = beta_reg # if number, new colat / if false, no new colat 
# gamma_new = gamma_star*(beta_reg+1)/(1+gamma_star*beta_reg) # if number, new own funds / if false, no new own funds. we need to also add beta reg to ensure LCR can be covered with additional cash (and we still match the solvency ratio despite this excess of stress) in the case of no payment shocks and no collateral creation
gamma_new = 3*gamma

# substitution of collateral
substitution = False

dic_default_value = {
    "nb_banks": nb_banks,
    "initial_deposits_size": initial_deposits_size,
    "alpha_init": alpha_init,
    "alpha": alpha,
    "beta_init": beta_init,
    "beta_reg": beta_reg,
    "beta_star": beta_star,
    "beta_new": beta_new,
    "gamma_init": gamma_init,
    "gamma": gamma,
    "gamma_star": gamma_star, 
    "gamma_new": gamma_new, 
    "collateral_value": collateral_value,
    "initialization_method": initialization_method,
    "alpha_pareto": alpha_pareto,
    "shocks_method": shocks_method,
    "shocks_law": shocks_law,
    "shocks_vol": shocks_vol,
    "LCR_mgt_opt": LCR_mgt_opt,
    "min_repo_trans_size": min_repo_trans_size,
    "loan_tenor": loan_tenor, # for money creation 
    "loan_period": loan_period,
    "new_loans_vol": new_loans_vol,
    "new_loans_mean": new_loans_mean,
    "end_repo_period": end_repo_period,
    "nb_steps": nb_steps,
    "path_results": f"{path}runs/",
    "dump_period": dump_period,
    "plot_period": plot_period,
    "cp_option": cp_option,
    "heavy_plot": heavy_plot,  # False to avoid the number of linux node to explode
    "substitution":substitution,
    "learning_speed":learning_speed,
    "check_leverage_opt":check_leverage_opt,
}

# special dictionary of parameters ranges 
list_dic_range = [
    # {"nb_banks":  [_ for _ in np.arange(50, 550,50)]*20},
    {"alpha_pareto": [_ for _ in np.logspace(0,2,100)*0.4]*50,},
    # {"gamma_new": [_ for _ in np.arange(1, 2,0.01)*gamma_star]*50,
    #  "gamma_init": [_ for _ in np.arange(1, 2,0.01)*gamma_star]*50
    #  },
    # {"learning_speed": [_ for _ in np.arange(0.00, 1.01, 0.01)]*50},
    {"learning_speed": [_ for _ in np.logspace(-5,0,100)]*50 + [0]*50}, 
    # {"shocks_vol":  [_ for _ in np.arange(0, 0.30, 0.003)]*50},
]

# initialize the path
# fct.delete_n_init_path(path)

# run parameter space analysis

In [2]:


# build list of the dic_args to be tested
list_dic_args = fct.build_args(dic_default_value, list_dic_range)

# open a cluster
client, cluster = new_launch_cluster(
    task_memory=19,
    job_walltime="30:00:00",
    max_cpu=len(list_dic_args),
)

# run with dask distributed
dld_obj = [
    dask.delayed(dyn.single_run)(**dic_args) for dic_args in list_dic_args
]
futures = client.compute(dld_obj)


/mnt/live/user/vlecoz/conda_env/money_markets/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38239 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://10.70.211.9:38239/status,
Dashboard: http://10.70.211.9:38239/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.70.211.9:45317,Workers: 0
Dashboard: http://10.70.211.9:38239/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [ ]:
! squeue -u vlecoz -h -t pending,running -r -O "state" | uniq -c

In [69]:
from tqdm import tqdm

# store error types
sr_dask_finished = pd.Series()
for i, future in enumerate(tqdm(futures)):
    if future.status == "finished":
        sr_dask_finished.loc[i] = str(future.result())

/opt/tmp/tmp/ipykernel_199299/868845186.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sr_dask_finished = pd.Series()
100%|██████████| 15000/15000 [00:00<00:00, 1341604.86it/s]


In [70]:
sr_dask_finished.dropna().apply(lambda x: x[:18]).value_counts()

Series([], dtype: int64)

In [59]:
from tqdm import tqdm

# store error types
sr_dask_errors = pd.Series()
for i, future in enumerate(tqdm(futures)):
    if future.status == "error":
        sr_dask_errors.loc[i] = str(future.exception())

/opt/tmp/tmp/ipykernel_199299/2045360933.py:4: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sr_dask_errors = pd.Series()
100%|██████████| 15000/15000 [00:00<00:00, 43426.40it/s]


In [60]:
# look at the different types of dask errors
sr_dask_errors.dropna().apply(lambda x: x[:300]).value_counts()

RecursionError('maximum recursion depth exceeded in comparison')    426
dtype: int64

In [5]:
# check the logs saved, look for what appended to the died workers
import os
sr_logs =pd.Series()
path_logs = f"./dask_logs/D2024-06_21_T18-05-55.472605/"
files = os.listdir(path_logs)
for file in tqdm(files):
    sr_logs.loc[file]= open(f"{path_logs}/{file}", "r").read()

/opt/tmp/tmp/ipykernel_214199/1202213659.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sr_logs =pd.Series()
100%|██████████| 332/332 [01:22<00:00,  4.03it/s]


In [167]:
sr_bool = sr_logs.str.contains('tcp://10.67.203.13:44700', na=False, regex=True)
sr_bool.loc[sr_bool]

dask-worker-12488587.err    True
dtype: bool

In [7]:
15000/320

46.875

In [ ]:
client.shutdown()

In [4]:
if "client" in globals():
    client.close()
    cluster.close()
    client.shutdown()

# collect results

In [3]:
# collect results into df_network_sensitivity
df_network_sensitivity, sr_errors = fct.get_df_network_sensitivity(dic_default_value["path_results"])

/mnt/live/user/vlecoz/Secured_interbank_network/functions.py:91: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sr_errors = pd.Series(index=index)
100%|██████████| 101/101 [16:52<00:00, 10.02s/it]


In [11]:
df_network_sensitivity["network density-1"].loc["learning_speed",1]

3099         NaN
4999    0.673902
3999    0.727084
3599    0.653908
2799    0.061489
99      0.596856
1099    0.722480
2899    0.644740
1799    0.671217
2299    0.755748
2699    0.676046
4299    0.822884
299     0.277098
4799    0.649833
799          NaN
699     0.018013
999     0.165425
1399         NaN
3699    0.761614
3499    0.745107
3199    0.689993
499     0.547527
2999    0.800513
1499    0.758601
399     0.738148
2599    0.688518
4699    0.619834
1999    0.005515
3899         NaN
2099         NaN
1699         NaN
899     0.794993
4599    0.684465
3799         NaN
1299         NaN
199          NaN
4399         NaN
1899    0.710317
3299    0.755488
599          NaN
4199    0.690610
1599    0.539292
2499         NaN
1199    0.693746
3399    0.698979
2399         NaN
2199    0.730789
4499         NaN
4099    0.645848
4899         NaN
Name: network density-1, dtype: float64

In [44]:
 # look at the different types of errors
sr_bool_error = sr_errors.str.contains("ERROR", na=False, regex=True)
sr_errors[sr_bool_error].apply(lambda x: (x.split("***ERROR***:"))[1][:25]).value_counts()

 securities reused negati    16
 bank 43 has not enough c     9
 bank 19 has not enough c     8
 bank 45 has not enough c     8
 bank 42 has not enough c     8
 bank 20 has not enough c     8
 bank 46 has not enough c     7
 bank 10 has not enough c     7
 bank 6 has not enough co     7
 bank 31 has not enough c     6
 bank 14 has not enough c     6
 bank 11 has not enough c     6
 bank 1 has not enough co     6
 bank 5 has not enough co     5
 bank 21 has not enough c     5
 bank 48 has not enough c     5
 bank 38 has not enough c     5
 bank 37 has not enough c     4
 bank 32 has not enough c     4
 bank 35 has not enough c     4
 bank 22 has not enough c     4
 bank 9 has not enough co     4
 bank 13 has not enough c     4
 bank 4 has not enough co     4
 bank 2 has not enough co     4
 bank 7 has not enough co     4
 bank 26 has not enough c     4
 bank 24 has not enough c     4
 bank 40 has not enough c     3
 bank 8 has not enough co     3
 bank 17 has not enough c     3
 bank 36

In [45]:
sr_errors[sr_errors.str.contains("assets don't", na=False, regex=True)]

Series([], dtype: object)

# plot results

In [2]:
# plot the sensitivity
df_network_sensitivity = pd.read_csv(f"{dic_default_value['path_results']}df_network_sensitivity.csv", index_col=(0,1,2))
gx.plot_all_sensitivities(df_network_sensitivity,path=path)

100%|██████████| 18/18 [00:11<00:00,  1.57it/s]
